# EDA

## Marta ---------------------------------------------------------

put all of your code between here and the next person's name only

## Jesse ---------------------------------------------------------

put all of your code between here and the next person's name only

## CM ---------------------------------------------------------

put all of your code between here and the next person's name only

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
fire_df = pd.read_pickle('./data/fire_pickle.pkl')

In [3]:
fire_df

,FIRE_CODE,FIRE_NAME,COMPLEX_NAME,FIRE_YEAR,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,STAT_CAUSE_CODE,STAT_CAUSE_DESCR,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME
0,BJ8K,FOUNTAIN,None,2005,2453403.5,33,1300,9.0,Miscellaneous,2453403.5,33.0,1730,0.10,A,40.036944,-121.005833,CA,63,063,Plumas
1,AAC0,PIGEON,None,2004,2453137.5,133,0845,1.0,Lightning,2453137.5,133.0,1530,0.25,A,38.933056,-120.404444,CA,61,061,Placer
2,A32W,SLACK,None,2004,2453156.5,152,1921,5.0,Debris Burning,2453156.5,152.0,2024,0.10,A,38.984167,-120.735556,CA,17,017,El Dorado
3,None,DEER,None,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,1400,0.10,A,38.559167,-119.913333,CA,3,003,Alpine
4,None,STEVENOT,None,2004,2453184.5,180,1600,1.0,Lightning,2453189.5,185.0,1200,0.10,A,38.559167,-119.933056,CA,3,003,Alpine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880460,None,ODESSA 2,None,2015,2457291.5,269,1726,13.0,Missing/Undefined,2457291.5,269.0,1843,0.01,A,40.481637,-122.389375,CA,None,None,None
1880461,None,None,None,2015,2457300.5,278,0126,9.0,Miscellaneous,NaN,NaN,None,0.20,A,37.617619,-120.938570,CA,None,None,None
1880462,None,None,None,2015,2457144.5,122,2052,13.0,Missing/Undefined,NaN,NaN,None,0.10,A,37.617619,-120.938570,CA,None,None,None
1880463,None,None,None,2015,2457309.5,287,2309,13.0,Missing/Undefined,NaN,NaN,None,2.00,B,37.672235,-120.898356,CA,None,None,None


In [4]:
fire_df.isnull().sum()

FIRE_CODE           300047
FIRE_NAME            64196
COMPLEX_NAME        514477
FIRE_YEAR                0
DISCOVERY_DATE           0
DISCOVERY_DOY            0
DISCOVERY_TIME      138738
STAT_CAUSE_CODE          0
STAT_CAUSE_DESCR         0
CONT_DATE           138864
CONT_DOY            138864
CONT_TIME           164573
FIRE_SIZE                0
FIRE_SIZE_CLASS          0
LATITUDE                 0
LONGITUDE                0
STATE                    0
COUNTY              285678
FIPS_CODE           285678
FIPS_NAME           285678
dtype: int64

In [5]:
# fire name NA for nulls
# fire code null to NONE
# DISCOVERY TIME : DROP
# CONT_DATE placeholder for nulls like 999999 
# Math it out subtract containment date from discovery date for tot_time_burn
# drop county and fibs name and Discovery_DOY and CONT DOY. 
# set all nulls in fibs_code to None

#make all columns lower_case. 

In [10]:
fire_df['FIPS_CODE'].unique()

array(['063', '061', '017', '003', '005', None, '027', '069', '037',
       '053', '089', '049', '009', '023', '043', '051', '039', '021',
       '025', '013', '031', '047', '510', '007', '057', '001', '011',
       '091', '081', '019', '015', '035', '006', '087', '065', '033',
       '029', '095', '075', '059', '055', '077', '093', '085', '045',
       '041', '079', '028', '115', '107', '067', '097', '105', '083',
       '071', '073', '109', '111', '099', '103', '012', '113', '101'],
      dtype=object)

## Kira ---------------------------------------------------------

put all of your code between here and the next person's name only